Baixando e extraindo o dataset:

Importando as bibliotecas necessárias.

In [14]:
!pip install tensorflow_addons
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import glob, warnings
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import os

warnings.filterwarnings('ignore')
print('TensorFlow Version ' + tf.__version__)

TensorFlow Version 2.7.0


In [15]:
from google.colab import drive
drive._mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [16]:
IMAGE_SIZE = 224
BATCH_SIZE = 16
EPOCHS = 100

diretorio = 'dataset_aumentado'

TRAIN_PATH = os.path.join(diretorio, 'train')
TEST_PATH = os.path.join(diretorio, 'test')
VALIDATION_PATH = os.path.join(diretorio, 'validation')

classes={
    0:'CD',
    1:'MF',
    2:'MP',
    3:'MR',
    4:'MG',
    5:'MT',
    6:'RG',
    7:'TA',
    8:'TB',
    9:'TR',
    10:'TC',
    11:'TN'
}

In [17]:
!pip install --quiet vit-keras

from vit_keras import vit

In [18]:
import tensorflow.keras.backend as K
import math
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

#------------------------
#step decay para o SGD
def step_decay1(epoch):
   initial_lrate = 1e-4
   flattern_factor = initial_lrate ** 2.25
   epochs_drop = 5.0
   #drop modelado como modelado no artigo
   drop = initial_lrate **(flattern_factor/epochs_drop)
   
   lrate = initial_lrate * math.pow(drop,  
           math.floor((epoch)/epochs_drop))
   return lrate

#step decay
def step_decay2(epoch):
   initial_lrate = 1e-6
   flattern_factor = initial_lrate ** 2.25
   epochs_drop = 5.0
   #drop modelado como modelado no artigo
   drop = initial_lrate **(flattern_factor/epochs_drop)
   
   lrate = initial_lrate * math.pow(drop,  
           math.floor((epoch)/epochs_drop))
   return lrate

In [19]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=vit.preprocess_inputs)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=vit.preprocess_inputs)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=vit.preprocess_inputs)

train_set = train_datagen.flow_from_directory(TRAIN_PATH,
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              class_mode='categorical',
                                              batch_size=BATCH_SIZE)

val_set = train_datagen.flow_from_directory(VALIDATION_PATH,
                                              target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                              class_mode='categorical',
                                              batch_size=BATCH_SIZE)

test_set = test_datagen.flow_from_directory(TEST_PATH, 
                                            target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                            class_mode='categorical',
                                            batch_size=1,
                                            shuffle=False)

Found 6718 images belonging to 12 classes.
Found 132 images belonging to 12 classes.
Found 72 images belonging to 12 classes.


In [20]:
vit_model = vit.vit_b16(
        image_size = IMAGE_SIZE,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = len(classes))

In [21]:
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout, BatchNormalization, GlobalMaxPooling2D

x = tf.keras.layers.Flatten()(vit_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.25)(x)
output = Dense(len(classes), activation='softmax')(x)
model = tf.keras.models.Model(vit_model.input, output) 


for layer in range(len(vit_model.layers)):
    vit_model.layers[layer].treinable=False

In [ ]:
learning_rate = 1e-4

optimizer = tfa.optimizers.RectifiedAdam(learning_rate = learning_rate)

#model.compile(optimizer = optimizer, 
              #loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2), 
              #metrics = ['accuracy'])

model.compile(optimizer='sgd', loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2),
           metrics=['accuracy', f1_m,precision_m, recall_m])

lrate = tf.keras.callbacks.LearningRateScheduler(step_decay1)


STEP_SIZE_TRAIN = train_set.n // train_set.batch_size
STEP_SIZE_VALID = val_set.n // val_set.batch_size

#reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_accuracy',
                                                 #factor = 0.2,
                                                 #patience = 2,
                                                 #verbose = 1,
                                                 #min_delta = 1e-4,
                                                 #min_lr = 1e-6,
                                                 #mode = 'max')

earlystopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy',
                                                 min_delta = 1e-4,
                                                 patience = 3,
                                                 mode = 'max',
                                                 restore_best_weights = True,
                                                 verbose = 1)

checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath = '/content/gdrive/MyDrive/model_best11v2.h5',
                                                  monitor = 'val_accuracy', 
                                                  verbose = 1, 
                                                  save_best_only = True,
                                                  save_weights_only = True,
                                                  mode = 'max')

#callbacks = [earlystopping, reduce_lr, checkpointer]

callbacks = [earlystopping, lrate, checkpointer]

model.fit(x = train_set,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_set,
          validation_steps = STEP_SIZE_VALID,
          epochs = EPOCHS,
          callbacks = callbacks)

model.load_weights('/content/gdrive/MyDrive/model_best11v2.h5')

learning_rate = 1e-6

lrate2 = tf.keras.callbacks.LearningRateScheduler(step_decay2)

#descongela as camadas
for layer in range(len(vit_model.layers)):
    vit_model.layers[layer].treinable=True

  

#model.compile(optimizer = optimizer, 
              #loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2), 
              #metrics = ['accuracy'])

model.compile(optimizer='sgd', loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing = 0.2),
           metrics=['accuracy', f1_m,precision_m, recall_m])

callbacks = [earlystopping, lrate2, checkpointer]

model.fit(x = train_set,
          steps_per_epoch = STEP_SIZE_TRAIN,
          validation_data = val_set,
          validation_steps = STEP_SIZE_VALID,
          epochs = EPOCHS,
          callbacks = callbacks)

model.load_weights('/content/gdrive/MyDrive/model_best11v2.h5')


Epoch 1/100
419/419 [==============================] - ETA: 0s - loss: 2.7575 - accuracy: 0.0861 - f1_m: 0.0020 - precision_m: 0.0167 - recall_m: 0.0010
Epoch 00001: val_accuracy improved from -inf to 0.10156, saving model to /content/gdrive/MyDrive/model_best11v2.h5
419/419 [==============================] - 681s 2s/step - loss: 2.7575 - accuracy: 0.0861 - f1_m: 0.0020 - precision_m: 0.0167 - recall_m: 0.0010 - val_loss: 2.5009 - val_accuracy: 0.1016 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/100
419/419 [==============================] - ETA: 0s - loss: 2.6627 - accuracy: 0.0997 - f1_m: 2.8078e-04 - precision_m: 0.0024 - recall_m: 1.4916e-04
Epoch 00002: val_accuracy improved from 0.10156 to 0.13281, saving model to /content/gdrive/MyDrive/model_best11v2.h5
419/419 [==============================] - 662s 2s/step - loss: 2.6627 - accuracy: 0.0997 - f1_m: 2.8078e-04 - precision_m: 0.0024 - recall_m: 1.4916e-04 - val_loss: 2.

In [ ]:
classes={
    'Codega':'CD',
    'Malvasia Fina':'MF',
    'Malvasia Preta':'MP',
    'Malvasia Rei':'MR',
    'Moscatel Galego':'MG',
    'Mourisco Tinto':'MT',
    'Rabigato':'RG',
    'Tinta Amarela':'TA',
    'Tinta Barroca':'TB',
    'Tinta Roriz':'TR',
    'Tinto Cao':'TC',
    'Touriga Nacional':'TN'
}

model.load_weights('/content/gdrive/MyDrive/model_best11v2.h5')
from sklearn import metrics
import itertools

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def confusion_matrix(test_data_generator, model):
  test_data_generator.reset()
  predictions = model.predict(test_data_generator, steps=test_set.samples)
  # Get most likely class
  predicted_classes = np.argmax(predictions, axis=1)
  true_classes = test_data_generator.classes
  class_labels = list(test_data_generator.class_indices.keys())
  print(class_labels)
  class_labels = [classes[x] for x in class_labels]  

  report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
  cm = metrics.confusion_matrix(true_classes, predicted_classes)
  print(report)
  plot_confusion_matrix(cm, class_labels)


confusion_matrix(test_set, model)

In [ ]:
dot_img_file = 'model_1.png'
tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

model.summary()